In [24]:
import os
import json
import pandas as pd
import folium
import gpxpy
import gpxpy.gpx


### Helper functions

In [26]:
# Read and combine all json files
def read_json_files(folder_path):
    combined_edges = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                edges = data["data"]["ntb_findCabins"]["edges"]
                combined_edges.extend(edges)
    return combined_edges


In [17]:
# Convert data into pandas dataframe
def convert_to_dataframe(data):
    records = []
    for item in data:
        node = item["node"]
        geometry = node["geometry"]
        record = {
            "id": node["id"],
            "name": node["name"],
            "serviceLevel": node["serviceLevel"],
            "dntCabin": node["dntCabin"],
            "ownername": node["owner"]["name"],
            "latitude": geometry["coordinates"][1],  # longitude
            "longitude": geometry["coordinates"][0],  # latitude
            "height": geometry["coordinates"][2],  # height
            "area name": node["areas"][0]["name"] if node["areas"] else ""  # area name
        }
        records.append(record)
    df = pd.DataFrame(records)
    return df


### Read json files

In [9]:
folder_path = './data/'
combined_data = read_json_files(folder_path)

### Convert to pandas

In [18]:
df = convert_to_dataframe(combined_data)

### Check length
Was 646 on 2024-03-15, should be "similar" at any later point

In [11]:
len(combined_data)

646

### Show table

In [19]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,area name
0,1033,Gressvasshytta,no-service,True,Hemnes Turistforening,66.048069,14.425802,605,Indre Helgeland
1,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli"
2,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya"
3,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland
4,1083,Damtjønna,emergency shelter,True,Trondhjems Turistforening,62.802600,9.891300,678,Trollheimen
...,...,...,...,...,...,...,...,...,...
641,10483,Angeltjønnhytta,self-service,True,DNT Nord-Trøndelag,63.462446,12.058861,516,Nord-Trøndelag
642,10478,Prestøyhytta,self-service,True,DNT Nord-Trøndelag,63.289152,11.341622,623,Sylan / Bealjehkh
643,10477,Kvitfjellhytta,self-service,True,DNT Nord-Trøndelag,63.343956,11.255825,500,Sylan / Bealjehkh
644,10475,Ramsjøhytta,self-service,True,Trondhjems Turistforening,63.163908,11.722157,771,Sylan / Bealjehkh


### Generate map

In [22]:
m = folium.Map(location=[60.472, 8.468], zoom_start=6)

# Add markers for each point in the DataFrame
for i, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        tooltip=f"Height: {row['height']} meters"
    ).add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file

In [23]:
m

### Export to GPX file

In [25]:
gpx = gpxpy.gpx.GPX()

# Create a GPX waypoint for each row in the DataFrame
for i, row in df.iterrows():
    waypoint = gpxpy.gpx.GPXWaypoint(latitude=row['latitude'], longitude=row['longitude'], elevation=row['height'])
    waypoint.name = row['name']
    gpx.waypoints.append(waypoint)

# Save the GPX to a file
with open('points.gpx', 'w') as f:
    f.write(gpx.to_xml())


#### Statistics for nerds